# Financial Chatbot Prototype (Notebook)

This notebook builds a simplified chatbot that answers a few **predefined queries** 
about Microsoft, Tesla, and Apple using the CSV you built in Task 1.  

**Features**
- Loads `10K_key_financials_MSFT_TSLA_AAPL.csv`
- Computes YoY growth, leverage, and cash conversion
- Answers fixed queries like:
  - *What is the latest total revenue by company?*
  - *How has net income changed over the last year?*
  - *Which company has the strongest cash conversion?*
  - *What is the leverage ratio last year?*
  - *What is the 3-year revenue sum by company?*
  - *Show summary*

In [3]:
# 1) Load data (using your fixed path) and compute derived metrics

from pathlib import Path
import pandas as pd
import numpy as np

CSV_PATH = Path("/Users/aish/10K_key_financials_MSFT_TSLA_AAPL.csv")

df = pd.read_csv(CSV_PATH)

# Coerce numerics
for c in ["Fiscal Year","Total Revenue","Net Income","Total Assets","Total Liabilities","CFO"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Sort and derive metrics
df = df.sort_values(["Company","Fiscal Year"]).reset_index(drop=True)
df["Revenue Growth (%)"]    = df.groupby("Company")["Total Revenue"].pct_change()*100
df["Net Income Growth (%)"] = df.groupby("Company")["Net Income"].pct_change()*100
df["CFO Growth (%)"]        = df.groupby("Company")["CFO"].pct_change()*100
df["Leverage (Liab/Assets)"]   = df["Total Liabilities"] / df["Total Assets"]
df["Cash Conversion (CFO/NI)"] = df["CFO"] / df["Net Income"]

df.head()

/var/folders/6b/hcmgcg951v75_fd0v_wnwk640000gn/T/ipykernel_92101/4218334254.py:17: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["Revenue Growth (%)"]    = df.groupby("Company")["Total Revenue"].pct_change()*100
/var/folders/6b/hcmgcg951v75_fd0v_wnwk640000gn/T/ipykernel_92101/4218334254.py:18: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["Net Income Growth (%)"] = df.groupby("Company")["Net Income"].pct_change()*100
/var/folders/6b/hcmgcg951v75_fd0v_wnwk640000gn/T/ipykernel_92101/4218334254.py:19: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change

,Company,Fiscal Year,Total Revenue,Net Income,Total Assets,Total Liabilities,CFO,Revenue Growth (%),Net Income Growth (%),CFO Growth (%),Leverage (Liab/Assets),Cash Conversion (CFO/NI)
0,Apple,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Microsoft,2022,NaN,72738.0,364840.0,198298.0,89035.0,NaN,NaN,NaN,0.54352,1.224051
4,Microsoft,2023,211915.0,72361.0,411976.0,205753.0,87582.0,NaN,-0.518299,-1.631942,0.49943,1.210348


## Helper functions
Utilities to format numbers and grab latest-year rows per company.

In [4]:
import math

def latest_year_rows(df: pd.DataFrame) -> pd.DataFrame:
    latest = df.groupby("Company")["Fiscal Year"].max().rename("LatestFY")
    out = df.merge(latest, on="Company")
    return out[out["Fiscal Year"] == out["LatestFY"]].copy()

def fmt_money(x):
    return "N/A" if pd.isna(x) else f"${x:,.0f}M"

def fmt_pct(x):
    return "N/A" if pd.isna(x) else f"{x:.1f}%"

def safe_ratio(x):
    return "N/A" if pd.isna(x) else f"{x:.2f}"

def yoy(df: pd.DataFrame, col: str) -> pd.DataFrame:
    latest = latest_year_rows(df)[["Company","Fiscal Year",f"{col} Growth (%)"]]
    return latest.rename(columns={f"{col} Growth (%)":"YoY (%)"})

def three_year_sum(df: pd.DataFrame, col: str) -> pd.DataFrame:
    s = df.groupby("Company")[col].sum(min_count=1).rename("3-yr Sum")
    return s.reset_index()


In [7]:
# --- Patch: align growth-column names with metric names ---

GROWTH_COL = {
    "Total Revenue": "Revenue Growth (%)",
    "Net Income": "Net Income Growth (%)",
    "CFO": "CFO Growth (%)",
}

def yoy(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """Return the latest YoY % for a given metric name (e.g., 'Total Revenue', 'Net Income', 'CFO')."""
    growth_col = GROWTH_COL.get(col)
    if growth_col is None or growth_col not in df.columns:
        raise KeyError(f"Growth column for '{col}' not found. Expected one of: {list(GROWTH_COL.values())}")
    latest = latest_year_rows(df)[["Company", "Fiscal Year", growth_col]]
    return latest.rename(columns={growth_col: "YoY (%)"})

## Chatbot logic
Exact matches first, then a simple case-insensitive contains fallback.

In [10]:
# Supported queries
PREDEFINED = {
    "What is the latest total revenue by company?": "latest_revenue",
    "How has net income changed over the last year?": "net_income_yoy",
    "Which company has the strongest cash conversion?": "best_cash_conversion",
    "What is the leverage ratio last year?": "latest_leverage",
    "What is the 3-year revenue sum by company?": "revenue_sum_3yr",
    "Show summary": "show_summary",
    "help": "help",
}

# Answer functions
# --- FIX: use iterrows() instead of itertuples() so r['col'] works ---

def answer_latest_revenue(df):
    lt = latest_year_rows(df)[["Company","Fiscal Year","Total Revenue"]]
    lines = []
    for _, r in lt.iterrows():
        lines.append(f"{r['Company']} FY{int(r['Fiscal Year'])}: {fmt_money(r['Total Revenue'])}")
    return "Latest total revenue by company:\n- " + "\n- ".join(lines)

def answer_net_income_yoy(df):
    y = yoy(df, "Net Income")
    lines = []
    for _, r in y.iterrows():
        lines.append(f"{r['Company']} FY{int(r['Fiscal Year'])}: {fmt_pct(r['YoY (%)'])}")
    return "Net income YoY change (latest FY):\n- " + "\n- ".join(lines)

def answer_best_cash_conversion(df):
    lt = latest_year_rows(df)[["Company","Fiscal Year","Cash Conversion (CFO/NI)","CFO","Net Income"]]
    lt = lt.sort_values("Cash Conversion (CFO/NI)", ascending=False)
    best = lt.iloc[0]
    return (
        "Strongest cash conversion (CFO/NI, latest FY):\n"
        f"- {best['Company']} FY{int(best['Fiscal Year'])}: "
        f"{safe_ratio(best['Cash Conversion (CFO/NI)'])} "
        f"(CFO {fmt_money(best['CFO'])} vs NI {fmt_money(best['Net Income'])})"
    )

def answer_latest_leverage(df):
    lt = latest_year_rows(df)[["Company","Fiscal Year","Leverage (Liab/Assets)","Total Liabilities","Total Assets"]]
    lines = []
    for _, r in lt.iterrows():
        lines.append(
            f"{r['Company']} FY{int(r['Fiscal Year'])}: "
            f"{safe_ratio(r['Leverage (Liab/Assets)'])} "
            f"(Liab {fmt_money(r['Total Liabilities'])} / Assets {fmt_money(r['Total Assets'])})"
        )
    return "Leverage (Liabilities/Assets, latest FY):\n- " + "\n- ".join(lines)

def answer_revenue_sum_3yr(df):
    s = three_year_sum(df, "Total Revenue")
    lines = []
    for _, r in s.iterrows():
        lines.append(f"{r['Company']}: {fmt_money(r['3-yr Sum'])}")
    return "3-year revenue sums:\n- " + "\n- ".join(lines)

def answer_show_summary(df):
    rev = yoy(df,"Total Revenue").set_index("Company")["YoY (%)"].to_dict()
    ni  = yoy(df,"Net Income").set_index("Company")["YoY (%)"].to_dict()
    cfo = yoy(df,"CFO").set_index("Company")["YoY (%)"].to_dict()
    companies = sorted(df["Company"].dropna().unique())
    lines = []
    for co in companies:
        lines.append(
            f"{co}: Revenue {fmt_pct(rev.get(co))}, "
            f"Net Income {fmt_pct(ni.get(co))}, "
            f"CFO {fmt_pct(cfo.get(co))} (YoY, latest FY)."
        )
    return "Summary (YoY, latest FY):\n- " + "\n- ".join(lines)

def help_text():
    opts = [k for k in PREDEFINED.keys() if k != "help"]
    return "I can answer these predefined queries:\n" + "\n".join(f"- {o}" for o in opts)

DISPATCH = {
    "latest_revenue":        answer_latest_revenue,
    "net_income_yoy":        answer_net_income_yoy,
    "best_cash_conversion":  answer_best_cash_conversion,
    "latest_leverage":       answer_latest_leverage,
    "revenue_sum_3yr":       answer_revenue_sum_3yr,
    "show_summary":          answer_show_summary,
    "help":                  lambda df: help_text(),
}

def ask(query: str) -> str:
    # Exact match first
    if query in PREDEFINED:
        return DISPATCH[PREDEFINED[query]](df)
    # Simple contains fallback (case-insensitive)
    q = query.lower()
    for key, tag in PREDEFINED.items():
        if key != "help" and key.lower() in q:
            return DISPATCH[tag](df)
    return "Sorry, I can only provide information on predefined queries. Type 'help' to see them."


## Quick tests
Run to verify responses for each predefined query.

In [11]:
tests = [
    "Show summary",
    "What is the latest total revenue by company?",
    "How has net income changed over the last year?",
    "Which company has the strongest cash conversion?",
    "What is the leverage ratio last year?",
    "What is the 3-year revenue sum by company?",
]
for t in tests:
    print("Q:", t)
    print(ask(t))
    print("-"*80)


Q: Show summary
Summary (YoY, latest FY):
- Apple: Revenue N/A, Net Income N/A, CFO N/A (YoY, latest FY).
- Microsoft: Revenue 15.7%, Net Income 21.8%, CFO 35.4% (YoY, latest FY).
- Tesla: Revenue 0.9%, Net Income -52.7%, CFO 12.6% (YoY, latest FY).
--------------------------------------------------------------------------------
Q: What is the latest total revenue by company?
Latest total revenue by company:
- Apple FY2024: N/A
- Microsoft FY2024: $245,122M
- Tesla FY2024: $97,690M
--------------------------------------------------------------------------------
Q: How has net income changed over the last year?
Net income YoY change (latest FY):
- Apple FY2024: N/A
- Microsoft FY2024: 21.8%
- Tesla FY2024: -52.7%
--------------------------------------------------------------------------------
Q: Which company has the strongest cash conversion?
Strongest cash conversion (CFO/NI, latest FY):
- Tesla FY2024: 2.10 (CFO $14,923M vs NI $7,091M)
------------------------------------------------

## Interactive chat (in-notebook)
Type a question (or `help`) and press Enter. Type `quit` to exit.

In [12]:
while True:
    try:
        user = input("Ask (or 'help' / 'quit'): ").strip()
    except (EOFError, KeyboardInterrupt):
        print("\nBye!")
        break
    if user.lower() in {"quit","exit"}:
        print("Bye!")
        break
    print(ask(user))
    print()


Ask (or 'help' / 'quit'):  help


I can answer these predefined queries:
- What is the latest total revenue by company?
- How has net income changed over the last year?
- Which company has the strongest cash conversion?
- What is the leverage ratio last year?
- What is the 3-year revenue sum by company?
- Show summary



Ask (or 'help' / 'quit'):  Show Summary


Summary (YoY, latest FY):
- Apple: Revenue N/A, Net Income N/A, CFO N/A (YoY, latest FY).
- Microsoft: Revenue 15.7%, Net Income 21.8%, CFO 35.4% (YoY, latest FY).
- Tesla: Revenue 0.9%, Net Income -52.7%, CFO 12.6% (YoY, latest FY).



Ask (or 'help' / 'quit'):  What is the latest total revenue by company?


...

3-year revenue sums:
- Apple: N/A
- Microsoft: $457,037M
- Tesla: $275,925M



Ask (or 'help' / 'quit'):  help


I can answer these predefined queries:
- What is the latest total revenue by company?
- How has net income changed over the last year?
- Which company has the strongest cash conversion?
- What is the leverage ratio last year?
- What is the 3-year revenue sum by company?
- Show summary



Ask (or 'help' / 'quit'):  quit


Bye!


## Short documentation

**How it works.**  
- Loads `10K_key_financials_MSFT_TSLA_AAPL.csv`.  
- Computes YoY growth for Revenue, Net Income, and CFO; plus Leverage (Liab/Assets) and Cash Conversion (CFO/NI).  
- Answers six predefined queries using simple matching (`ask(query)`).

**Limitations.**  
- Predefined questions only (no free-form NLP).  
- Missing values (e.g., Apple if not yet filled) show as `N/A`.  
- No authentication or persistence.